In [10]:
import pandas as pd
import subprocess
import pysam
import os

# Chip-seq pipeline

In [102]:
sample_name = "GSM4087690" #paired_one_run
#sample_name = "GSM1666208" #single_one_run
#sample_name = "GSM4256992" #paired_multiple_runs
#sample_name = "GSM4491173" #single_multiple_runs


In [103]:
n_threads = str(20)

In [104]:
factor = "H3K27ac"

In [105]:
out_dir = "results_paired_one_run"

In [106]:
ref_path = "./reference_data/"
fasta_folder = f"./{out_dir}/fastq_files/"
fastqc_folder = f"./{out_dir}/fastqc/"
genome_reference = "./genome_reference/"
bowtie_results = f"./{out_dir}/bowtie2/"
intermediate_bams = f"./{out_dir}/bowtie2/intermediate_bams/"
qualimap_folder = f"./{out_dir}/qualimap/"
macs2_out = f"./{out_dir}/macs2_out/"

In [107]:
subprocess.call(" ".join(["mkdir", "-p", fasta_folder]), shell=True)
subprocess.call(" ".join(["mkdir", "-p", fastqc_folder]), shell=True)
subprocess.call(" ".join(["mkdir", "-p", intermediate_bams]), shell=True)

0

In [108]:
align_sam = f"./{out_dir}/bowtie2/{sample_name}_m_unsorted.sam"
align_bam = f"./{out_dir}/bowtie2/{sample_name}_m_unsorted.bam"
align_n_sorted = f"./{out_dir}/bowtie2/{sample_name}_m_name_sorted.bam"
align_fixmate = f"./{out_dir}/bowtie2/{sample_name}_m_fixmate_sorted.bam"
align_filtered = f"./{out_dir}/bowtie2/{sample_name}_m_filtered_sorted.bam"
align_sorted = f"./{out_dir}/bowtie2/{sample_name}_sorted.bam"

In [109]:
info_query = ["esearch", "-db", "sra", "-query", sample_name, "|", "efetch", "-format", "runinfo"]

In [110]:
broad = ["H3F3A", "H3K27ME3", "H3K36ME3", "H3K4ME1", "H3K79ME2", "H3K79ME3", "H3K9ME1", "H3K9ME2" , "H4K20ME1"]

In [112]:
proc = subprocess.Popen(" ".join(info_query), shell=True, stdout=subprocess.PIPE)
serviceList  = proc.communicate()[0].decode('utf-8')

In [113]:
factor_type = "--broad" if factor.upper() in broad else ""

In [114]:
file_meta = open(f"./{out_dir}/"+ sample_name +".meta", "w")
srr_id=[]
values = serviceList.rstrip().split('\n')
for runs in range(1, len(values)):
    for attributes in range(0, len(values[0].split(","))):
        #print(str(values[0].split(",")[attributes]) + '\t' + str(values[runs].split(",")[attributes]))
        file_meta.write(str(values[0].split(",")[attributes]) + '\t' + str(values[runs].split(",")[attributes]) + "\n")
        if values[0].split(",")[attributes] == 'Run':
            srr_id.append(values[runs].split(",")[attributes])       
        if values[0].split(",")[attributes] == "LibraryLayout":
            layout = values[1].split(",")[attributes]
        if values[0].split(",")[attributes] == "ScientificName":
            organism = values[1].split(",")[attributes]
file_meta.close()

In [115]:
if organism.upper() == 'HOMO SAPIENS':
    ref = "hg38" 
else: 
    ref = "mm10"

In [116]:
if layout.upper() == 'SINGLE':
    print("LAYOUT: SINGLE")
    fastq_name = fasta_folder + sample_name + ".fastq"
    
    for srr in srr_id:
        download_query = " ".join(["parallel-fastq-dump", "--sra-id", srr, "--threads", n_threads, "--outdir", fasta_folder])
        subprocess.call(download_query, shell=True)        
    
    merge_files = " ".join(["cat", fasta_folder + "*.fastq", ">",  fastq_name])
    subprocess.call(merge_files, shell=True) 

    fastqc_query = " ".join(["fastqc", fastq_name, "--extract"])
    subprocess.call(fastqc_query, shell=True)

    mv_fastqc_query = " ".join(['mv', fasta_folder + '*_fastqc*', fastqc_folder])
    subprocess.call(mv_fastqc_query, shell=True)

    fastqc_df = pd.read_csv(fastqc_folder + sample_name + "_fastqc/fastqc_data.txt", nrows=10, sep="\t")
    encoding = str(fastqc_df[fastqc_df["##FastQC"]=="Encoding"].iloc[:, 1]).split("\n")[0].split("  ")[-1]
    quality = "" if "Sanger" in encoding else "--phred64"

    bowtie2 = " ".join(['bowtie2', '-p', n_threads, quality, '-q', '-D', '20', '-R', '3', '-N', '1', '-L', '20', '-x', ref_path + ref, '-U', fastq_name, '-S', align_sam, '2> ./' + out_dir + '/log_file_test.txt'])
    subprocess.call(bowtie2, shell=True)

    drop_srr = " ".join(["rm", fasta_folder + "*SRR*"]) 
    subprocess.call(drop_srr, shell=True)

    bam_conversion = " ".join(["samtools", "view", "-h", "-S", "-b", "-@", n_threads, "-o", align_bam, align_sam])
    subprocess.call(bam_conversion, shell=True)

    bam_name_sorting = " ".join(["samtools", "sort", "-n", "-@", n_threads, "-o", align_n_sorted, align_bam])
    subprocess.call(bam_name_sorting, shell=True)

    fixmate = " ".join(["samtools", "fixmate", "-@", n_threads, align_n_sorted, align_fixmate])
    subprocess.call(fixmate, shell=True)

    bam_sorting = " ".join(["samtools", "sort", "-@", n_threads, "-o", align_sorted, align_fixmate])
    subprocess.call(bam_sorting, shell=True)
    
    filtered = " ".join(["samtools", "markdup","-r", "-@", n_threads, align_sorted, align_filtered])
    subprocess.call(filtered, shell=True)

    index_creation  = " ".join(["samtools", "index", "-@", n_threads, align_sorted])
    subprocess.call(index_creation, shell=True)

    move_intermediate  = " ".join(["mv", bowtie_results + "*_m_*", intermediate_bams])
    subprocess.call(move_intermediate, shell=True)

    qualimap = " ".join(["qualimap", "bamqc", "-c", "-nt", n_threads, "-sdmode 2", "--java-mem-size=4G", "-bam", align_sorted, "-outdir", qualimap_folder])
    subprocess.call(qualimap, shell=True)

    macs2 = " ".join(["macs2", "callpeak", "-q", "0.01", "--keep-dup 1", "--extsize=150", "--nomodel", "-f", "BAM", "-g", "2.9e+9", factor_type, "-t", align_sorted, "-n", sample_name, "--outdir", macs2_out])
    subprocess.call(macs2, shell=True)
    
if layout.upper() == 'PAIRED':
    print("LAYOUT: PAIRED")
    fastq_1 = fasta_folder + sample_name + "_1" + ".fastq"
    fastq_2 = fasta_folder + sample_name + "_2" + ".fastq"
    for srr in srr_id:
        download_query = ["parallel-fastq-dump", "-F", "--split-files", "--sra-id", srr, "--threads", n_threads, "--outdir", fasta_folder]
        subprocess.call(" ".join(download_query), shell=True)

    r1 = " ".join(["cat", fasta_folder + "*_1.fastq", ">",  fastq_1])
    subprocess.call(r1, shell=True) 

    r2 = " ".join(["cat", fasta_folder + "*_2.fastq", ">",  fastq_2])
    subprocess.call(r2, shell=True)

    fastqc_query = " ".join(["fastqc", fasta_folder + "*GSM*", "--extract"])
    subprocess.call(fastqc_query, shell=True)

    mv_fastqc_query = " ".join(['mv', fasta_folder + '*_fastqc*', fastqc_folder])
    subprocess.call(mv_fastqc_query, shell=True)

    fastqc_df = pd.read_csv(fastqc_folder + sample_name + "_1_fastqc/fastqc_data.txt", nrows=10, sep="\t")
    encoding = str(fastqc_df[fastqc_df["##FastQC"]=="Encoding"].iloc[:, 1]).split("\n")[0].split("  ")[-1]
    quality = "" if "Sanger" in encoding else "--phred64"

    bowtie2 = " ".join(['bowtie2', '-p', n_threads, quality, '-q', '-D', '20', '-R', '3', '-N', '1', '-L', '20', '-x', ref_path + ref, '-1', fastq_1, '-2', fastq_2, '-S', align_sam, '2> ./' + out_dir + '/log_file_test.txt'])
    subprocess.call(bowtie2, shell=True)

    drop_srr = " ".join(["rm", fasta_folder + "*SRR*"]) 
    subprocess.call(drop_srr, shell=True)

    bam_conversion = " ".join(["samtools", "view", "-h", "-S", "-b", "-@", n_threads, "-o", align_bam, align_sam])
    subprocess.call(bam_conversion, shell=True)

    fixmate = " ".join(["samtools", "fixmate", "-@", n_threads, "-m", align_bam, align_fixmate])
    subprocess.call(fixmate, shell=True)

    bam_sorting = " ".join(["samtools", "sort", "-@", n_threads, "-o", align_sorted, align_fixmate])
    subprocess.call(bam_sorting, shell=True)

    filtered = " ".join(["samtools", "markdup","-r", "-@", n_threads, "-s", align_sorted, align_filtered])
    subprocess.call(filtered, shell=True)

    index_creation  = " ".join(["samtools", "index", "-@", n_threads, align_sorted])
    subprocess.call(index_creation, shell=True)

    move_intermediate  = " ".join(["mv", bowtie_results + "*_m_*", intermediate_bams])
    subprocess.call(move_intermediate, shell=True)

    qualimap = " ".join(["qualimap", "bamqc", "-nt", n_threads, "-sdmode 2", "--java-mem-size=4G", "-ip", "-bam", align_sorted, "-outdir", qualimap_folder])
    subprocess.call(qualimap, shell=True)

    macs2 = " ".join(["macs2", "callpeak", "-q", "0.01","-c", "--keep-dup 1", "--extsize=150", "--nomodel", "-f", "BAMPE", "-g", "2.9e+9", factor_type, "-t", align_sorted, "-n", sample_name, "--outdir", macs2_out])
    subprocess.call(macs2, shell=True)


bed_file_type = "broad" if factor_type.split("-")[-1] == 'broad' else "narrow"
bed_file = macs2_out + sample_name + "_peaks." + bed_file_type +  "Peak"

blacklisted = " ".join(["bedtools", "intersect", "-v", "-a", bed_file, "-b", genome_reference + "hg38-blacklist.v2.bed", ">", bed_file + "_unblacked.bed"])
subprocess.call(blacklisted, shell=True)

# PCR bottleneck coefficient (PBC) & Non-Redundant Fraction (NRF)

In [146]:
sam = intermediate_bams + align_bam.split("bowtie2/")[-1]

In [147]:
inputfile = pysam.AlignmentFile(sam, "rb")
lines = []
for line in inputfile:
    string = str(line.reference_name) + "_" + str(line.reference_start) + "_" + str(line.reference_end)
    lines.append(string)

In [148]:
df = pd.DataFrame({'col':lines})

In [149]:
total_reads = df.shape[0]

In [150]:
distinct_uniquely_mapping_reads = df.drop_duplicates().shape[0]

In [151]:
df_2 = df['col'].value_counts().to_frame('counts').reset_index()

In [152]:
N_dist = df_2.shape[0]

In [153]:
N1 = df_2[df_2['counts'] == 1].shape[0]

In [154]:
N2 = df_2[df_2['counts'] == 2].shape[0]

In [155]:
print("NFR: " + str(round(distinct_uniquely_mapping_reads/total_reads, 2)))

NFR: 0.83


In [156]:
print("PBC1: " + str(round(N1/N_dist, 2)))

PBC1: 0.85


In [157]:
print("PBC2: " + str(round(N1/N2, 2)))

PBC2: 7.67


# Frip Score (fraction of reads in peaks)

In [165]:
import deeptools.countReadsPerBin as crb
import pysam

bam_file = align_sorted
bed_file_type = "broad" if factor_type.split("-")[-1] == 'broad' else "narrow"
bed_file = macs2_out + sample_name + "_peaks." + bed_file_type +  "Peak"

cr = crb.CountReadsPerBin([bam_file], bedFile=bed_file, numberOfProcessors=int(n_threads))
reads_at_peaks = cr.run()

total = reads_at_peaks.sum(axis=0)
bam = pysam.AlignmentFile(bam_file)

frip_score = round((float(total[0]) / bam.mapped) * 100, 2) 
print("Frip score: " + str(frip_score) + "%")

Frip score: 46.25%
